In [1]:
import re
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
import pandas as pd

# Import Data
df = pd.read_csv("C:/Kuliah/Bangkit/Capstone/Sentiment/data_final.csv")

# Select specific columns using double square brackets
df = df[['at',"comment", "label"]]

# Cut the last 10 characters in each cell of the specified column
df['at'] = df['at'].apply(lambda x: x[:-9] if len(x) > 10 else '')

df

,at,comment,label
0,2023-11-18,ribet mengganti alamat surel aktif buka bantua...,negative
1,2023-11-14,kecewa aplikasi bintang kali transaksi paylate...,negative
2,2023-11-14,daftar harga langsung token listrik harga tota...,negative
3,2023-11-06,jagan beli pulsa bualapak top up transfer puls...,negative
4,2023-11-12,pembelian dibatalkan keterangannya pembeli bat...,negative
...,...,...,...
27530,2020-05-18,costumer terhormat tolong order order cancel k...,negative
27531,2019-04-03,membantu banget kadang greget kupon motornya k...,negative
27532,2022-03-10,menerima kode otp aktivasi gopay nomor aplikas...,negative
27533,2019-02-18,ship membantu raport nilai pelit tip cas mang ...,positive


In [6]:
# Assuming 'label' is the column containing 'positive' and 'negative'
negative_rows = df[df['label'] == 'negative']

# Keep up to 10471 rows of 'negative' and all rows of 'positive'
df = pd.concat([negative_rows.head(10254), df[df['label'] != 'negative']])


In [7]:
# Assuming 'label' is the column containing 'positive' and 'negative'
label_counts = df['label'].value_counts()

# Display the counts
print(label_counts)


label
negative    10254
positive    10254
Name: count, dtype: int64


In [8]:
reviews = df['comment'].values
labels = df['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [9]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [ ]:
#BUAT TEST AJA JGN DI RUN

vocab_size = 15000
oov_tok = ''
embedding_dim = 50
max_length = 200
padding_type = 'post'
trunc_type = 'post'

# Tokenize sentences
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

# Convert test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

# Perform PCA to estimate embedding dimension
#embedding_dim_candidate = 50  # You can start with a value and adjust as needed
pca = PCA(n_components=embedding_dim)
pca_result = pca.fit_transform(train_padded)

# Plot the explained variance ratio to help choose an appropriate dimension
plt.plot(range(1, embedding_dim + 1), pca.explained_variance_ratio_)
plt.title('Explained Variance Ratio by Number of Components')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.show()


In [10]:
vocab_size = 30000 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 70
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

# Ensure all elements in train_sentences are strings
train_sentences = [str(sentence) for sentence in train_sentences]
test_sentences = [str(sentence) for sentence in test_sentences]

# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length,truncating=trunc_type)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length,truncating=trunc_type)

In [ ]:
# Import Keras and its layers
import keras
from keras.layers import Embedding, Bidirectional, LSTM, Dense, BatchNormalization
from keras.regularizers import l2

# Define the hyperparameters
lstm_units = 64 # The number of units in the LSTM layer
dense_units = 24 # The number of units in the first dense layer
l2_lambda = 0.01 # The L2 regularization factor
learning_rate = 0.01 # The learning rate for the optimizer

# Create the sequential model
model = keras.Sequential()

# Add the embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

# Add the bidirectional LSTM layer with dropout and recurrent dropout
model.add(Bidirectional(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)))

# Add the first dense layer with relu activation and batch normalization
model.add(Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_lambda)))
model.add(BatchNormalization())

# Add the second dense layer with linear activation
model.add(Dense(1, activation='linear', kernel_regularizer=l2(l2_lambda)))

# Compile the model with mean squared error loss and RMSprop optimizer
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate), metrics=['accuracy'])

# Print the model summary
model.summary()


In [11]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 70)           2100000   
                                                                 
 bidirectional (Bidirection  (None, 128)               69120     
 al)                                                             
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 2172241 (8.29 MB)
Trainable params: 2172241 (8.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
from keras.callbacks import ModelCheckpoint

num_epochs = 5
checkpoint = ModelCheckpoint('C:/Kuliah/Bangkit/Capstone/Sentiment/test4_model.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, 
                    validation_split=0.1,callbacks=[checkpoint])

Epoch 1/5


433/433 [==============================] - ETA: 0s - loss: 0.2788 - accuracy: 0.8816
Epoch 1: val_accuracy improved from -inf to 0.91423, saving model to C:/Kuliah/Bangkit/Capstone/Sentiment\test4_model.h5
433/433 [==============================] - 44s 95ms/step - loss: 0.2788 - accuracy: 0.8816 - val_loss: 0.2363 - val_accuracy: 0.9142
Epoch 2/5
  1/433 [..............................] - ETA: 34s - loss: 0.1220 - accuracy: 0.9062

C:\Users\Oriza Sativa\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


433/433 [==============================] - ETA: 0s - loss: 0.1441 - accuracy: 0.9477
Epoch 2: val_accuracy did not improve from 0.91423
433/433 [==============================] - 39s 90ms/step - loss: 0.1441 - accuracy: 0.9477 - val_loss: 0.2523 - val_accuracy: 0.9071
Epoch 3/5
433/433 [==============================] - ETA: 0s - loss: 0.0935 - accuracy: 0.9686
Epoch 3: val_accuracy did not improve from 0.91423
433/433 [==============================] - 38s 88ms/step - loss: 0.0935 - accuracy: 0.9686 - val_loss: 0.2745 - val_accuracy: 0.9142
Epoch 4/5
433/433 [==============================] - ETA: 0s - loss: 0.0655 - accuracy: 0.9784
Epoch 4: val_accuracy did not improve from 0.91423
433/433 [==============================] - 36s 83ms/step - loss: 0.0655 - accuracy: 0.9784 - val_loss: 0.3244 - val_accuracy: 0.9077
Epoch 5/5
433/433 [==============================] - ETA: 0s - loss: 0.0486 - accuracy: 0.9838
Epoch 5: val_accuracy did not improve from 0.91423
433/433 [==================

In [13]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

161/161 [==============================] - 4s 20ms/step
Accuracy of prediction on test set :  0.9028671737858397


In [14]:
# Assuming you have already initialized 'tokenizer' and 'max_length'

# Test sentence
test_sentence = "ui bagus"

# Convert the test sentence to a sequence
test_sequence = tokenizer.texts_to_sequences([test_sentence])

# Pad the sequence
padded_test = pad_sequences(test_sequence, padding='post', maxlen=max_length)

# Get the label based on probability 1 if p >= 0.5 else 0
prediction = model.predict(padded_test)

# Convert probability to label
pred_label = 1 if prediction >= 0.5 else 0

# Display the result
print("Test sentence: ", test_sentence)
print("Predicted sentiment: ", "Positive" if pred_label == 1 else "Negative")


1/1 [==============================] - 0s 19ms/step
Test sentence:  ui bagus
Predicted sentiment:  Positive


In [ ]:
from keras.models import load_model
# Memuat model yang telah disimpan
loaded_model = load_model('best_model.h5')  # Ganti 'best_model.h5' dengan nama file model yang sesuai

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(test_labels, pred_labels)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=['negatif', 'positif'], yticklabels=['negatif', 'positif'])
plt.title('Confusion Matrix')
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.show()

In [ ]:
from wordcloud import WordCloud

text_data = ' '.join(df['title'].astype(str))

wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords='spotify').generate(text_data)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud')
plt.show()

In [ ]:
count_sentiment = df['label'].value_counts()

plt.figure(figsize=(6, 6))
plt.pie(count_sentiment, labels=count_sentiment.index, autopct='%1.1f%%', colors=['pink', 'orange'], startangle=90)
plt.title('Distribusi Sentimen Hasil Model')
plt.show()

In [ ]:
# Convert 'at' column to datetime if it's not already in datetime format
df['at'] = pd.to_datetime(df['at'])

# Create a new column 'positive' with 1 for positive values and 0 for others
df['positive'] = (df['label'] == 'positive').astype(int)

# Create a new column 'negative' with 1 for negative values and 0 for others
df['negative'] = (df['label'] == 'negative').astype(int)

# Assuming that 'label' can have values other than 'positive' and 'negative'
# You can set a default value (e.g., 0) for cases where 'label' is neither 'positive' nor 'negative'
df['positive'] = df['positive'].where(df['label'].isin(['positive', 'negative']), 0)
df['negative'] = df['negative'].where(df['label'].isin(['positive', 'negative']), 0)

# Optionally, you can set the 'at' column as the index for a time series
df.set_index('at', inplace=True)

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Plot positive values in green
ax.plot(at, label, label='Positive', color='green')

# Plot negative values in red
ax.plot(at, label, label='Negative', color='red')

# Customize the plot
ax.set_title('Positive and Negative Trends in Time Series')
ax.set_xlabel('Time')
ax.set_ylabel('Values')
ax.legend()

# Show the plot
plt.show()

In [ ]:
# Assuming you have already initialized 'tokenizer' and 'max_length'

# Load the data
df = pd.read_csv('[LABELLED]_LinkedIn.csv', sep=',')
df = df[["text_data"]]

# Convert to a sequence
sequences = tokenizer.texts_to_sequences(df["text_data"])

# Pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get labels based on probability 1 if p >= 0.5 else 0
prediction = model.predict(padded)

# Convert probabilities to labels
pred_labels = [1 if p >= 0.5 else 0 for p in prediction]

# Create a new DataFrame with the original text data and predicted sentiments
result_df = pd.DataFrame({"text_data": df["text_data"], "predicted_sentiment": pred_labels})

# Display predictions
for i in range(len(result_df)):
    print(result_df["text_data"].iloc[i])
    print("Predicted sentiment: ", "Positive" if result_df["predicted_sentiment"].iloc[i] == 1 else "Negative")
    print()

# Save the results to a new CSV file
result_df.to_csv('predicted_sentiments.csv', index=False)
